# Problem Set 5 Solutions

PH - 531 E&M

Author:  Guillermo Fidlago


Use the finite element method described in Section 2.12 of Jackson’s textbook to find the potential inside a square of side $a$. 
The potential at the top of the square is one volt $(V(x,y=a)=1$ Volt). 

The potential on the other three sides is zero. There is no charge inside the square. 
Compare your result with the exact result which you can obtain using separation of variables

For the analytical method we have to solve the Laplace Equation with Separation of Variables



$$ \nabla^2 V = 0 $$

\begin{equation*}
\frac{\partial^2 V}{ \partial x^2}  + \frac{\partial^2 V}{\partial y^2} = 0
\end{equation*}

If we assume a Solution of the form 
$$
    X(x)Y(y) = V(x,y)
$$

we end with the following expression 
$$
\underbrace{\frac{X''}{X}}_{\alpha^2} + \underbrace{\frac{Y''}{Y}}_{\beta^2}= 0
$$
This can only be true for two constant terms that add to zero and in this case $\alpha^2 = -\beta^2$.
Due to the symmetry of this problem we choose the following two second order ODE's.

\begin{gather*}
\frac{d^2 V}{d x^2} = -\alpha^2 X \to A\cos \alpha x + B \sin \alpha x \\
\frac{d^2 V}{d y^2} = \alpha^2 Y \to C e^{\alpha y} + D e^{-\alpha y}
\end{gather*}